In [2]:
## 한국은행 API이용한 일별 데이터 불러오기

from urllib.request import urlopen
import json
import pandas as pd
import datetime
import timedelta

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
# 통계항목 및 API코드 기준 엑셀파일은 별도 작성함
ecos_std =pd.read_excel('한국은행_API로드_기준리스트.xlsx', converters={'통계항목코드' : str})
ecos_std

,No,통계자료명,통계코드,통계항목명,통계항목코드,단위
0,1,주식시장(일별),064Y001,KOSPI_거래량,0002000,만주
1,2,주식시장(일별),064Y001,KOSPI_거래대금,0003000,억원
2,3,주식시장(일별),064Y001,KOSPI_외국인순매수,0030000,억원
3,4,주식시장(일별),064Y001,KOSPI_거래량시간외,0087000,만주
4,5,주식시장(일별),064Y001,KOSPI_거래대금시간외,0088000,억원
5,6,주식시장(일별),064Y001,KOSPI_시가총액,0183000,억원
6,7,주식시장(일별),064Y001,KOSDAQ_거래량,0090000,만주
7,8,주식시장(일별),064Y001,KOSDAQ_거래대금,0091000,억원
8,9,주식시장(일별),064Y001,KOSDAQ_외국인순매수,0113000,억원
9,10,시장금리(일별),060Y001,"콜금리(1일, 전체거래)",010101000,연%


In [4]:
# 현재 날짜 가져오기 , format = YYYYMMDD
today_iso = datetime.datetime.now().isoformat()[:10].replace('-','')

# 현재 날짜 가져오기 , format = YYYY-MM-DD
today = datetime.datetime.now()
today_s = str(today.year) + '-' +  str(today.month) + '-' +str(today.day)

# 19970101 ~ 현재 갯수 구하기
tot_len = len(pd.date_range("2003-1-2", today_s))

In [5]:
# 한국은행 ECOS API 데이터 로드
## 통계표 : 주식시장(일별)
from urllib.request import urlopen
from tqdm import tqdm_notebook
import json

key = '💛'

for i in tqdm_notebook(ecos_std.index):
    # url="http://ecos.bok.or.kr/api/StatisticSearch/"+key+"/json/kr/1/24/010Y002/MM/201401/201912/?/?/?/"
    url="http://ecos.bok.or.kr/api/StatisticSearch/" + key + "/json/kr/1/" + str(tot_len) + "/"  + str(ecos_std["통계코드"][i]) + "/DD/20000101/" + today_iso + "/" + str(ecos_std["통계항목코드"][i]) +"/?/?/"

    result = urlopen(url)
    html = result.read()
    data = json.loads(html)
    data = data["StatisticSearch"]["row"]
    data = pd.DataFrame(data)
    data["TIME"] = data["TIME"].astype(str)

    #컬럼 정리 : 
    # TIME => Date : YYYY-MM-DD
    # DATA_VALU => kr_bs_rate
    data["Date"] = data["TIME"].str[:4] + "-" + data["TIME"].str[4:6] + "-" + data["TIME"].str[6:]
    data = data.rename(columns={'DATA_VALUE' : ecos_std["통계항목명"][i]})
    data = data[["Date",ecos_std["통계항목명"][i]]]
    data = data.reset_index(drop=True)
    data = data.sort_values(["Date"])
    data.to_csv("D:\\시세모니터링\\융합분석\\DATA\\" + datetime.datetime.now().isoformat()[:10].replace("-","") + "\\"+ ecos_std["통계항목명"][i] + ".csv", index=False)
    

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


            Date KOSPI_거래량
0     2003-01-02     74286
1     2003-01-03    100575
2     2003-01-06     72180
3     2003-01-07     69012
4     2003-01-08     60487
...          ...       ...
4307  2020-06-05     84007
4308  2020-06-08     84291
4309  2020-06-09     80525
4310  2020-06-10     69521
4311  2020-06-11     81382

[4312 rows x 2 columns]
            Date KOSPI_거래대금
0     2003-01-02      13532
1     2003-01-03      22705
2     2003-01-06      15986
3     2003-01-07      18700
4     2003-01-08      14885
...          ...        ...
4307  2020-06-05     125834
4308  2020-06-08     129256
4309  2020-06-09     145235
4310  2020-06-10     126818
4311  2020-06-11     166430

[4312 rows x 2 columns]
            Date KOSPI_외국인순매수
0     2003-01-02          -60
1     2003-01-03         2539
2     2003-01-06         1535
3     2003-01-07         1331
4     2003-01-08          125
...          ...          ...
4311  2020-06-05         -720
4312  2020-06-08          114
4313  2020-06-09    

            Date 통안증권(91일)
0     2006-09-25      4.54
1     2006-09-26      4.55
2     2006-09-27      4.55
3     2006-09-28      4.55
4     2006-09-29      4.54
...          ...       ...
3397  2020-06-05     0.608
3398  2020-06-08      0.61
3399  2020-06-09     0.606
3400  2020-06-10     0.607
3401  2020-06-11     0.605

[3402 rows x 2 columns]
            Date 통안증권(1년)
0     2000-01-04     8.99
1     2000-01-05     9.04
2     2000-01-06     9.05
3     2000-01-07     9.04
4     2000-01-08     9.05
...          ...      ...
5114  2020-06-05    0.674
5115  2020-06-08    0.674
5116  2020-06-09    0.666
5117  2020-06-10    0.661
5118  2020-06-11    0.659

[5119 rows x 2 columns]
            Date 통안증권(2년)
0     2000-01-04     9.43
1     2000-01-05     9.51
2     2000-01-06     9.57
3     2000-01-07     9.57
4     2000-01-08     9.58
...          ...      ...
5198  2020-06-05    0.799
5199  2020-06-08    0.796
5200  2020-06-09    0.784
5201  2020-06-10    0.775
5202  2020-06-11    0.774

[